In [1]:
import sys
sys.path.insert(0, '/home/berhe/Desktop/Thesis_git/TLP_thesis/codes/')

In [2]:
import Scene_Clustering_Representations as scr
import os
import sklearn.cluster as cluster
from sklearn.manifold import TSNE
import evaluationMetrics as em
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pylab as plt

Directories to the subtitles and scene boundries/labels files

In [3]:
scenes_dir='/home/berhe/Desktop/LIMSI/TLP_thesis/Scenes/'
subtitle_dir='/home/berhe/Desktop/LIMSI/TLP_thesis/subtitles/'

subtitleFiles=[]
for fl in os.listdir(subtitle_dir):
    if '.en.srt' in fl:
       subtitleFiles.append(fl)

scenesegementedFiles=[]
for fl in os.listdir(scenes_dir):
    if 'Episode' in fl:
       scenesegementedFiles.append(fl)

subtitleFiles.sort()
scenesegementedFiles.sort()

Get all texts of  scenes from all the episodes

In [4]:
scene_textsA=[]
scene_lblsA=[]
startA=[]
endA=[]
for i in range(len(scenesegementedFiles)):
    scene_texts,scene_lbls ,start,end=scr.sceneTexts(scenes_dir+scenesegementedFiles[i],subtitle_dir+subtitleFiles[i])
    for i in scene_texts:
        if len(i)!=0:
            scene_textsA.append(i.decode("utf-8"))
    scene_lblsA=scene_lblsA+scene_lbls
    startA=startA+start
    endA=endA+end

In [5]:
scene_texts,scene_lbls ,start,end=scr.sceneTexts(scenes_dir+scenesegementedFiles[0],subtitle_dir+subtitleFiles[0])

In [6]:
len(scene_textsA),len(scene_lblsA)#total number of scenes in 9 episodes of GoT

(241, 274)

In [7]:
print(scene_textsA[0])

(Gate rumbling, chains rattling)
(Horse snorts)
(Rumbling, chain rattling)


# Topic Modelling 

In [8]:
import TopicModels as tm

Loading BokehJS ...

In [9]:
tokenized_data=tm.tokenizeDocuments(scene_textsA)

In [11]:
model_LDA=tm.buildLDA_model(tokenized_data,6)

3965 241


/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin ins

/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin ins

In [13]:
tm.print_topMF(model_LDA,6,6)

Topic :0: 0.016*"lord" + 0.013*"king" + 0.009*"man" + 0.007*"father" + 0.007*"lady" + 0.006*"boy"
####################################################################################################
Topic :1: 0.012*"lord" + 0.007*"king" + 0.007*"one" + 0.006*"come" + 0.006*"want" + 0.005*"men"
####################################################################################################
Topic :2: 0.011*"know" + 0.010*"king" + 0.010*"father" + 0.010*"man" + 0.010*"one" + 0.009*"lord"
####################################################################################################
Topic :3: 0.011*"lord" + 0.009*"would" + 0.007*"like" + 0.007*"man" + 0.006*"king" + 0.006*"get"
####################################################################################################
Topic :4: 0.010*"king" + 0.008*"lord" + 0.007*"see" + 0.006*"know" + 0.006*"men" + 0.006*"brother"
####################################################################################################
Topic :

In [ ]:
from gensim import corpora
diction=corpora.Dictionary(tokenized_data)
corpus=[diction.doc2bow(text) for text in s

In [12]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorize=CountVectorizer(min_df=5,max_df=0.9,lowercase=True,stop_words='english',token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vector=vectorize.fit_transform(scene_textsA)

<input>:2: DeprecationWarning: invalid escape sequence \-
<input>:2: DeprecationWarning: invalid escape sequence \-
<input>:2: DeprecationWarning: invalid escape sequence \-
<ipython-input-20-4a48e39673ff>:2: DeprecationWarning: invalid escape sequence \-
  vectorize=CountVectorizer(min_df=5,max_df=0.9,lowercase=True,stop_words='english',token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


In [21]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vector.T)
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorize.get_feature_names()
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
text_font_size="8pt", text_color="#66666",
source=source, text_align='center')
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [28]:
lda_model = LatentDirichletAllocation(n_topics=10, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vector)
 
text = scene_textsA[1]
x = lda_model.transform(vectorize.transform([text]))[0]
print(x, x.sum())

/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


[0.0076924  0.00769237 0.00769232 0.00769339 0.00769361 0.00769371
 0.00769242 0.00769236 0.93076492 0.00769249] 1.0


In [38]:
predicted_topics=[]
for tex in scene_textsA: 
    x = lda_model.transform(vectorize.transform([tex]))[0]
    x=[i for i in x]
    #print('the topic label: {}'.format((x.index(max(x))+1)))
    predicted_topics.append(x.index(max(x))+1)

In [39]:
len(predicted_topics)

241

In [23]:
text = "Stark"
x = lda_model.transform(vectorize.transform([text]))[0]
print(x, x.sum())

[0.05000018 0.05000204 0.54998998 0.05000009 0.05000516 0.05000169
 0.05000048 0.05000028 0.05000001 0.05000009] 1.0


In [24]:
import pyLDAvis

In [27]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vector, vectorize, mds='tsne')
panel

/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2       91.128883  181.784134       1        1  61.917741
1      -83.841759  242.081863       2        1   9.644511
5      105.688759 -141.396133       3        1   9.496643
4     -220.409088  -83.423630       4        1   8.795831
3      -70.580353   88.548668       5        1   3.764799
0      -65.023338  -61.879555       6        1   3.204196
8       62.508469   18.162415       7        1   1.292654
7     -227.553253   94.427505       8        1   1.202408
6      -62.825291 -216.145035       9        1   0.446092
9      214.650192   25.878180      10        1   0.235125, topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
328   Default  253.000000      lord  253.000000  30.0000  30.0000
278   Default  192.000000      king  192.000000  29.0000  29.0000
135   Default  161.000000       don  161.000000  28.0000  28.0000
291   Default   80.000000      lady   80.000000  27.0000  27.0000
344   Default  148.000000       man  148.000000  26.0000  26.0000
90    Default   84.000000      come   84.000000  25.0000  25.0000
358   Default  105.000000       men  105.000000  24.0000  24.0000
53    Default   83.000000       boy   83.000000  23.0000  23.0000
137   Default   61.000000  dothraki   61.000000  22.0000  22.0000
316   Default  100.000000      like  100.000000  21.0000  21.0000
239   Default   41.000000     horse   41.000000  20.0000  20.0000
487   Default   64.000000       ser   64.000000  19.0000  19.0000
287   Default  146.000000      know  146.000000  18.0000  18.0000
572   Default   84.000000      tell   84.000000  17.0000  17.0000
117   Default   42.000000      dead   42.000000  16.0000  16.0000
531   Default   36.000000    speaks   36.000000  15.0000  15.0000
303   Default   19.000000  laughter   19.000000  14.0000  14.0000
123   Default   99.000000       did   99.000000  13.0000  13.0000
579   Default   60.000000     think   60.000000  12.0000  12.0000
638   Default   41.000000     watch   41.000000  11.0000  11.0000
271   Default   55.000000      just   55.000000  10.0000  10.0000
310   Default   63.000000       let   63.000000   9.0000   9.0000
274   Default   34.000000  khaleesi   34.000000   8.0000   8.0000
199   Default   51.000000     grace   51.000000   7.0000   7.0000
211   Default   56.000000      hand   56.000000   6.0000   6.0000
249   Default   26.000000      hurt   26.000000   5.0000   5.0000
560   Default   34.000000     sword   34.000000   4.0000   4.0000
55    Default   24.000000      bran   24.000000   3.0000   3.0000
382   Default   17.000000    neighs   17.000000   2.0000   2.0000
522   Default   66.000000       son   66.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
196   Topic10    0.034348     gonna    4.982567   1.0756  -6.7040
95    Topic10    0.038009  commands    5.527356   1.0732  -6.6027
166   Topic10    0.041006    follow    5.975285   1.0712  -6.5268
309   Topic10    0.049725      legs    7.504758   1.0360  -6.3340
232   Topic10    0.079086       hit   12.814095   0.9650  -5.8700
552   Topic10    0.057001    summer    8.889925   1.0032  -6.1974
580   Topic10    0.045097  thinking    6.737527   1.0462  -6.4317
575   Topic10    0.095917     thank   18.365128   0.7981  -5.6770
551   Topic10    0.087866    stupid   16.870565   0.7953  -5.7647
221   Topic10    0.040435    having    5.973916   1.0573  -6.5408
153   Topic10    0.045624     enemy    7.189989   0.9928  -6.4201
50    Topic10    0.073346    bloody   16.688082   0.6255  -5.9453
568   Topic10    0.050790   talking    9.979964   0.7722  -6.3128
146   Topic10    0.040231      ears    6.156445   1.0222  -6.5459
28    Topic10    0.049877    battle   11.447930   0.6168  -6.3309
203   Topic10    0.045326   greyjoy    8.911348   0.7716  -6.4266
411   Topic10    0.053487  

In [ ]:
tfidf_vector,terms=scr.tfidf_Representation(scene_textsA)

In [ ]:
tfidf_vector.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tsne_init = 'pca'  # could also be 'random'
tsne_perplexity = 10.0
tsne_early_exaggeration = 4.0
tsne_learning_rate = 1000
random_state = 1
model_pca = TSNE(n_components=2, random_state=random_state, init=tsne_init, perplexity=tsne_perplexity,
         early_exaggeration=tsne_early_exaggeration, learning_rate=tsne_learning_rate)

transformed_vector = model_pca.fit_transform(tfidf_vector)

In [ ]:
transformed_vector.shape

tf_idf_clust=scr.plot_clusters(transformed_vector,cluster.MeanShift, (0.175,), {'cluster_all':False})

In [ ]:
scene_lblsA = [w.replace('NA', '11') for w in scene_lblsA]

In [ ]:
scene_lblsA=[int(i) for i in scene_lblsA]

In [ ]:
scene_lblsA=np.array(scene_lblsA)

In [ ]:
print('accuracy %f' %(em.adjusted_rand_score(scene_lblsA,tf_idf_clust)))
print('NMI %f' %(em.normalized_mutual_info_score(scene_lblsA,tf_idf_clust)))
print('Purity %f' %(em.purity_score(scene_lblsA,tf_idf_clust)))
print('recall %f' %(em.recall(scene_lblsA,tf_idf_clust,rType='micro')))
print('precision %f' %(em.precision(scene_lblsA,tf_idf_clust,rType='micro')))

Count Vector

In [ ]:
dtm,vocab =scr.count_Vector(training_data=scene_textsA)
dist_eclud,cos_sim=scr.distance_Measures(dtm)

In [ ]:
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
pos_ecludian=mds.fit_transform(dist_eclud)
pos_cos=mds.fit_transform(cos_sim)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pos_cos[:, 0], pos_cos[:, 1], pos_cos[:, 1])

In [ ]:
fcluster_res=scr.heirarchical_dendogram(dtm)

In [ ]:
print('accuracy %f' %(em.adjusted_rand_score(scene_lblsA,fcluster_res)))
print('NMI %f' %(em.normalized_mutual_info_score(scene_lblsA,fcluster_res)))
print('Purity %f' %(em.purity_score(scene_lblsA,fcluster_res)))
print('recall %f' %(em.recall(scene_lblsA,fcluster_res,rType='micro')))
print('precision %f' %(em.precision(scene_lblsA,fcluster_res,rType='micro')))

In [ ]:
plt.figure(figsize=(10,8))
plt.title('cosine similarity visualization')
plt.scatter(pos_ecludian[:,0],pos_ecludian[:,1],c=fcluster_res,cmap='prism')
plt.show()